In [1]:
# Ye packages install kar rahe hain
!pip install transformers accelerate gradio
!pip install -U bitsandbytes transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 86.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
import gradio as gr
from threading import Thread
import json
import os
from pathlib import Path

In [3]:
# --- PERMANENT MODEL STORAGE SETUP ---
# Model ko is folder mein save karenge (permanent storage)
MODEL_CACHE_DIR = "./saved_models/zephyr-7b"  # Local folder for permanent storage


In [4]:
# Check karo ke model already downloaded hai ya nahi
def is_model_downloaded():
    """Check if model files exist in cache directory"""
    if not os.path.exists(MODEL_CACHE_DIR):
        return False
    # Check essential files
    required_files = ["config.json", "pytorch_model.bin.index.json"]
    for file in required_files:
        if not os.path.exists(os.path.join(MODEL_CACHE_DIR, file)):
            return False
    return True

In [5]:
# --- 1. Model Setup with Caching ---
model_id = "HuggingFaceH4/zephyr-7b-beta"

if is_model_downloaded():
    print("✅ Model already downloaded! Loading from cache...")
    print(f"📁 Loading from: {MODEL_CACHE_DIR}")
    model_path = MODEL_CACHE_DIR  # Local path se load karo
else:
    print("⏬ Downloading model for the first time...")
    print("⚠️ This will take 2-3 minutes, but ONLY ONCE!")
    model_path = model_id  # Pehli baar HuggingFace se download hoga

# Tokenizer load karo (cache enabled)
print("Loading Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    cache_dir=MODEL_CACHE_DIR if model_path == model_id else None,
    local_files_only=is_model_downloaded()  # Agar downloaded hai to internet use mat karo
)

# Model load karo (cache enabled)
print("Loading Model...")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True,
    cache_dir=MODEL_CACHE_DIR if model_path == model_id else None,
    local_files_only=is_model_downloaded()
)

# Agar pehli baar download hua to save kar do
if not is_model_downloaded() and model_path == model_id:
    print("💾 Saving model for future use...")
    os.makedirs(MODEL_CACHE_DIR, exist_ok=True)
    model.save_pretrained(MODEL_CACHE_DIR)
    tokenizer.save_pretrained(MODEL_CACHE_DIR)
    print(f"✅ Model saved to {MODEL_CACHE_DIR}")

print("🎉 Model ready!")


⏬ Downloading model for the first time...
⚠️ This will take 2-3 minutes, but ONLY ONCE!
Loading Tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

💾 Saving model for future use...
✅ Model saved to ./saved_models/zephyr-7b
🎉 Model ready!


In [6]:
# --- 2. Memory System Setup ---
MEMORY_FILE = "chatbot_memory.json"

def load_memory():
    """Load memory from file"""
    if os.path.exists(MEMORY_FILE):
        try:
            with open(MEMORY_FILE, 'r', encoding='utf-8') as f:
                return json.load(f)
        except:
            return {}
    return {}

def save_memory(memory_data):
    """Save memory to file"""
    with open(MEMORY_FILE, 'w', encoding='utf-8') as f:
        json.dump(memory_data, f, ensure_ascii=False, indent=2)

def extract_key_info(message, response):
    """Extract important info from conversation"""
    key_info = {}
    if "mera naam" in message.lower() or "my name is" in message.lower():
        words = message.split()
        for i, word in enumerate(words):
            if word.lower() in ["naam", "name", "is"]:
                if i + 1 < len(words):
                    key_info["user_name"] = words[i + 1].strip(".,!?")
    return key_info


In [7]:
# --- 3. Prompt Formatting with Memory ---
def format_chat_prompt(message, chat_history, instruction, memory_context):
    """Format chat prompt with memory"""
    enhanced_instruction = instruction
    if memory_context:
        enhanced_instruction += f"\n\nImportant context: {memory_context}"

    prompt = f"<|system|>\n{enhanced_instruction}</s>\n"
    for user_msg, bot_msg in chat_history[-10:]:
        prompt += f"<|user|>\n{user_msg}</s>\n<|assistant|>\n{bot_msg}</s>\n"
    prompt += f"<|user|>\n{message}</s>\n<|assistant|>\n"
    return prompt


In [8]:

# --- 4. Chat Logic with Streaming and Memory ---
def respond(message, chat_history, instruction, temperature, memory_state):
    """Main response function"""
    current_memory = memory_state if memory_state else load_memory()

    memory_context = ""
    if current_memory.get("user_name"):
        memory_context += f"User's name is {current_memory['user_name']}. "

    formatted_prompt = format_chat_prompt(message, chat_history, instruction, memory_context)
    inputs = tokenizer([formatted_prompt], return_tensors="pt").to("cuda")

    streamer = TextIteratorStreamer(
        tokenizer,
        skip_prompt=True,
        skip_special_tokens=True
    )

    generate_kwargs = dict(
        inputs,
        streamer=streamer,
        max_new_tokens=1024,
        do_sample=True,
        temperature=temperature,
        top_p=0.95,
    )

    thread = Thread(target=model.generate, kwargs=generate_kwargs)
    thread.start()

    chat_history.append((message, ""))
    acc_text = ""

    for new_text in streamer:
        acc_text += new_text
        chat_history[-1] = (message, acc_text)
        yield "", chat_history, current_memory

    # Update memory
    new_info = extract_key_info(message, acc_text)
    if "user_name" in new_info:
        current_memory["user_name"] = new_info["user_name"]
    current_memory["conversation_count"] = current_memory.get("conversation_count", 0) + 1
    save_memory(current_memory)

    yield "", chat_history, current_memory

def reset_memory():
    """Reset memory"""
    if os.path.exists(MEMORY_FILE):
        os.remove(MEMORY_FILE)
    return {}, "Memory cleared! 🧹"


In [10]:
# --- 5. Gradio UI ---
custom_css = """
*:focus, *:active, *:focus-visible {
    outline: 0 !important;
    box-shadow: none !important;
}

.gradio-container {
    max-width: 1200px !important;
    margin: auto !important;
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', 'Roboto', sans-serif !important;
}

.chat-container {
    border-radius: 12px !important;
    border: 1px solid #e5e5e5 !important;
    box-shadow: 0 1px 3px rgba(0,0,0,0.1) !important;
}

.message.user, .message.bot {
    background: #f7f7f8 !important;
    color: #333 !important;
    border-radius: 18px !important;
    padding: 12px 18px !important;
    margin: 8px 0 !important;
    border: 1px solid #e5e5e5 !important;
}

.input-box textarea {
    border: none !important;
    background: transparent !important;
    font-size: 15px !important;
    padding: 8px 12px !important;
    resize: none !important;
    outline: none !important;
}

.model-selector-inline select {
    background: #f5f5f5 !important;
    border-radius: 20px !important;
    padding: 8px 14px !important;
    font-size: 13px !important;
    font-weight: 500 !important;
    color: #333 !important;
    cursor: pointer !important;
    border: none !important;
    outline: none !important;
    transition: all 0.2s ease !important;
}

.model-selector-inline select:hover {
    background: #ebebeb !important;
    transform: translateY(-1px) !important;
}

.send-button {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    color: white !important;
    border: none !important;
    outline: none !important;
    border-radius: 20px !important;
    padding: 8px 20px !important;
    font-size: 14px !important;
    font-weight: 600 !important;
    cursor: pointer !important;
    transition: all 0.2s ease !important;
    box-shadow: 0 2px 4px rgba(102, 126, 234, 0.2) !important;
}

.send-button:hover {
    transform: translateY(-2px) !important;
    box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4) !important;
}

.header-text {
    text-align: center !important;
    font-size: 28px !important;
    font-weight: 700 !important;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    -webkit-background-clip: text !important;
    -webkit-text-fill-color: transparent !important;
    margin-bottom: 10px !important;
}

.settings-panel {
    background: #fafafa !important;
    border-radius: 12px !important;
    padding: 20px !important;
    border: 1px solid #e5e5e5 !important;
}

button {
    transition: all 0.2s ease !important;
}

button:hover {
    transform: translateY(-1px) !important;
}
"""

with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:

    gr.HTML("""
        <div style="text-align: center; padding: 30px 20px 20px 20px;">
            <h1 class="header-text">🤖 AI Assistant</h1>
            <p style="color: #666; font-size: 13px;">Powered by Zephyr-7B with Memory</p>
        </div>
    """)

    with gr.Row():
        with gr.Column(scale=7):
            chatbot = gr.Chatbot(
                elem_classes="chat-container",
                height=480,
                show_label=False,
                avatar_images=(None, "🤖"),
                bubble_full_width=False,
            )

            with gr.Row(elem_id="input-row"):
                msg = gr.Textbox(
                    placeholder="Reply...",
                    show_label=False,
                    lines=1,
                    max_lines=5,
                    elem_classes="input-box",
                    container=False,
                    scale=8
                )

                with gr.Column(scale=2, min_width=220):
                    with gr.Row():
                        model_selector = gr.Dropdown(
                            choices=[" Zephyr-7B ", "Opus 4", "Haiku 4"],
                            value=" Zephyr-7B ",
                            label="",
                            show_label=False,
                            elem_classes="model-selector-inline",
                            container=False,
                            scale=2
                        )
                        submit_btn = gr.Button(
                            "Send",
                            elem_classes="send-button",
                            size="sm",
                            scale=1,
                            interactive=True
                        )

        with gr.Column(scale=3, elem_classes="settings-panel"):
            gr.Markdown("### ⚙️ Advanced Settings")

            temp_slider = gr.Slider(
                minimum=0.1,
                maximum=1.0,
                value=0.7,
                step=0.1,
                label="🌡️ Temperature",
                info="Higher = more creative"
            )

            system_msg = gr.Textbox(
                label="📝 System Prompt",
                value="You are a helpful AI assistant with memory.",
                lines=4
            )

            gr.Markdown("---")
            gr.Markdown("### 🧠 Memory")
            memory_status = gr.Textbox(
                label="Status",
                value="Active ✓",
                interactive=False,
                lines=1
            )

            with gr.Row():
                clear_btn = gr.Button("🗑️ Clear Chat", size="sm")
                reset_memory_btn = gr.Button("🔄 Reset Memory", size="sm", variant="stop")

    memory_state = gr.State(load_memory())

    def respond_and_clear(message, chat_history, instruction, temperature, memory_state):
        if not message or not message.strip():
            return "", chat_history, memory_state
        for update in respond(message, chat_history, instruction, temperature, memory_state):
            yield update

    submit_btn.click(
        respond_and_clear,
        [msg, chatbot, system_msg, temp_slider, memory_state],
        [msg, chatbot, memory_state]
    )

    msg.submit(
        respond_and_clear,
        [msg, chatbot, system_msg, temp_slider, memory_state],
        [msg, chatbot, memory_state]
    )

    clear_btn.click(lambda: ([], ""), None, [chatbot, msg])

    reset_memory_btn.click(
        reset_memory,
        outputs=[memory_state, memory_status]
    )

demo.queue().launch(share=True, show_error=True)

/tmp/ipython-input-1575118148.py:101: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-1575118148.py:101: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-1575118148.py:112: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-1575118148.py:112: DeprecationWarning: The 'bubble_full_width' parameter will be removed in Grad

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0031b305603f7cdc33.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
